In [9]:
def filter_by_moneyness(df, pct_cutoff=0.2):
    crit1 = (1-pct_cutoff)*df.Strike < df.Underlying
    crit2 = df.Underlying< (1+pct_cutoff)*df.Strike
    return (df.loc[crit1 & crit2].reset_index(drop=True))            

def options_chain(symbol):
    import datetime
    import re
    import pandas as pd
    import QuantLib as ql
    import pandas as pd
    import numpy as np
    import yfinance as yf
    from yahoo_fin.stock_info import get_quote_table
    from pylab import plt
    plt.style.use('dark_background')
    import pandas as pd

    info = get_quote_table(symbol)
    current_price = info["Quote Price"]
    tk = yf.Ticker(symbol)
    exps = tk.options
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    options['yte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['midpoint'] = (options['bid'] + options['ask']) / 2 
    options['spread'] =  (options['ask'] - options['bid'])
    options['spread_pct'] = (options['ask'] - options['bid'])/options['ask'] 
    options['Underlying'] = current_price 
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
    return pd.DataFrame({
            "Underlying": current_price,
            "Ticker": symbol,
            "Expiry": options["expirationDate"],
            "YTE": options["yte"],
            "DTE": options["dte"],
            "Call": options["CALL"],
            "IV": options["impliedVolatility"],        
            "Strike": options["strike"],
            "Last": options["lastPrice"],
            "Bid": options["bid"],
            "Ask": options["ask"],
            "Midpoint": options['midpoint'],
            "Spread": options['spread'],
            "Spread_Pct": options['spread_pct'],
            "AKA": options["contractSymbol"]})

In [10]:
df=options_chain("AAPL")
df=filter_by_moneyness(df)

Example

Vanilla Call Option


In [80]:
def filter_by_moneyness(df, pct_cutoff=0.2):
    crit1 = (1-pct_cutoff)*df.Strike < df.Underlying
    crit2 = df.Underlying< (1+pct_cutoff)*df.Strike
    return (df.loc[crit1 & crit2].reset_index(drop=True))            

def know_your_options(ticker, option="Call"):
    import utils 
    import QuantLib as ql 
    import pandas as pd
    import yfinance as yf
    import numpy as np
    import matplotlib.pyplot as plt

    from pandas.tseries.offsets import BDay
    end = pd.datetime.today().date()
    start = end - 252 * BDay() * 1

    import warnings
    plt.style.use('dark_background')
    warnings.simplefilter(action='ignore', category=FutureWarning)
                
    info = get_quote_table(ticker)
    current_price = info["Quote Price"]
    yield_re = re.compile(r"\((?P<value>(\d+\.\d+))%\)")
    try:
        dividend_rate = float(yield_re.search(info["Forward Dividend & Yield"])["value"])
    except (KeyError, ValueError, TypeError):
        dividend_rate = 0.0      
        
    def create_call(row):

        risk_free_rate = 0.001
        day_count = ql.Actual365Fixed()
        calendar = ql.UnitedStates()

        calculation_date = ql.Date.todaysDate()
        ql.Settings.instance().evaluationDate = calculation_date

        exercise = EuropeanExercise(ql.Date(expiration.day, expiration.month, expiration.year))
        payoff = PlainVanillaPayoff(ql.Option.Call, row["strike"])
        european_option = EuropeanOption(payoff,exercise)
        
        spot_handle = ql.QuoteHandle(ql.SimpleQuote(current_price))
        flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count))
        dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_rate, day_count))
        flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, calendar, row["impliedVolatility"], day_count))
        bsm_process = ql.BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)
            
        european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))

        return {"Underlying": current_price,
                "Ticker": ticker,
                "YTE": row["yte"],
                "DTE": row["dte"],
                "Strike": row["strike"],
                "Last": row["lastPrice"],
                "Bid": row["bid"],
                "Ask": row["ask"],
                "IV": row["impliedVolatility"],
                "NPV": european_option.NPV(),
                "Delta": european_option.delta(),
                "Gamma": european_option.gamma(),
                "Theta": european_option.theta() / 365,
                "AKA": row["contractSymbol"]}        
        
    def create_put(row):

        risk_free_rate = 0.001
        day_count = ql.Actual365Fixed()
        calendar = ql.UnitedStates()

        calculation_date = ql.Date.todaysDate()
        ql.Settings.instance().evaluationDate = calculation_date

        exercise = EuropeanExercise(ql.Date(expiration.day, expiration.month, expiration.year))
        payoff = PlainVanillaPayoff(ql.Option.Put, row["strike"])
        european_option = EuropeanOption(payoff,exercise)
        
        spot_handle = ql.QuoteHandle(ql.SimpleQuote(current_price))
        flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count))
        dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_rate, day_count))
        flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, calendar, row["impliedVolatility"], day_count))
        bsm_process = ql.BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)
            
        european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))

        return {"Underlying": current_price,
                "Ticker": ticker,
                "YTE": row["yte"],
                "DTE": row["dte"],
                "Strike": row["strike"],
                "Last": row["lastPrice"],
                "Bid": row["bid"],
                "Ask": row["ask"],
                "IV": row["impliedVolatility"],
                "NPV": european_option.NPV(),
                "Delta": european_option.delta(),
                "Gamma": european_option.gamma(),
                "Theta": european_option.theta() / 365,
                "AKA": row["contractSymbol"]}                
        
    options_= pd.DataFrame()       
    
    if option == "Call":
        
        tk = yf.Ticker(ticker)
        exps = tk.options
        
        for e in exps:
            opt = tk.option_chain(e)
            calls = pd.DataFrame().append(opt.calls)
            calls['expirationDate'] = e
            expiration =  pd.to_datetime(e) + datetime.timedelta(days = 1)
            calls['expirationDate'] = expiration
            calls['yte'] = (calls['expirationDate'] - datetime.datetime.today()).dt.days / 365
            calls['dte'] = (calls['expirationDate'] - datetime.datetime.today()).dt.days
            calls[['bid', 'ask', 'strike']] = calls[['bid', 'ask', 'strike']].apply(pd.to_numeric)
            calls = calls.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
            strike_price = calls["strike"]
            
            options = calls.apply(create_call, axis=1, result_type="expand")
            options_ = options_.append(options, ignore_index=True) 
    else:    
        
        tk = yf.Ticker(ticker)
        exps = tk.options
        
        for e in exps:
            opt = tk.option_chain(e)
            puts = pd.DataFrame().append(opt.calls)
            puts['expirationDate'] = e
            expiration =  pd.to_datetime(e) + datetime.timedelta(days = 1)
            puts['expirationDate'] = expiration
            puts['yte'] = (puts['expirationDate'] - datetime.datetime.today()).dt.days / 365
            puts['dte'] = (puts['expirationDate'] - datetime.datetime.today()).dt.days
            puts[['bid', 'ask', 'strike']] = puts[['bid', 'ask', 'strike']].apply(pd.to_numeric)
            puts = puts.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
            strike_price = puts["strike"]
            
            options = puts.apply(create_put, axis=1, result_type="expand")
            options_ = options_.append(options, ignore_index=True) 
    return  options_

In [83]:
options = filter_by_moneyness(know_your_options("AAPL", option="Call"))

In [84]:
options

,Underlying,Ticker,YTE,DTE,Strike,Last,Bid,Ask,IV,NPV,Delta,Gamma,Theta,AKA
0,119.989998,AAPL,0.016438,6,100.0,20.00,19.90,20.00,0.500005,18.455216,0.980531,0.002197,0.208085,AAPL210326C00100000
1,119.989998,AAPL,0.016438,6,105.0,15.38,14.90,15.10,0.527348,13.605755,0.941694,0.010857,0.150690,AAPL210326C00105000
2,119.989998,AAPL,0.016438,6,106.0,14.10,13.90,14.05,0.449224,12.541088,0.952695,0.010169,0.172215,AAPL210326C00106000
3,119.989998,AAPL,0.016438,6,108.0,12.17,11.95,12.15,0.466802,10.689813,0.915755,0.017501,0.129226,AAPL210326C00108000
4,119.989998,AAPL,0.016438,6,109.0,11.40,11.00,11.15,0.433599,9.697107,0.909225,0.020142,0.128290,AAPL210326C00109000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,119.989998,AAPL,1.991781,727,125.0,21.79,21.60,21.85,0.355292,0.024643,0.001471,0.000070,0.000155,AAPL230317C00125000
365,119.989998,AAPL,1.991781,727,130.0,20.00,19.90,20.05,0.354956,0.019108,0.001164,0.000057,0.000119,AAPL230317C00130000
366,119.989998,AAPL,1.991781,727,135.0,18.45,18.35,18.50,0.356208,0.015623,0.000963,0.000048,0.000096,AAPL230317C00135000
367,119.989998,AAPL,1.991781,727,140.0,16.93,16.85,17.05,0.357001,0.012672,0.000792,0.000040,0.000077,AAPL230317C00140000


In [76]:
def custom_describe(df, nidx=3, nfeats=20):
    ''' Concat transposed topN rows, numerical desc & dtypes '''

    print(df.shape)
    nrows = df.shape[0]
    
    rndidx = np.random.randint(0,len(df),nidx)
    dfdesc = df.describe().T

    for col in ['mean','std']:
        dfdesc[col] = dfdesc[col].apply(lambda x: np.round(x,2))
 
    dfout = pd.concat((df.iloc[rndidx].T, dfdesc, df.dtypes), axis=1, join='outer')
    dfout = dfout.loc[df.columns.values]
    dfout.rename(columns={0:'dtype'}, inplace=True)
    
    # add count nonNAN, min, max for string cols
    nan_sum = df.isnull().sum()
    dfout['count'] = nrows - nan_sum
    dfout['min'] = df.min().apply(lambda x: x[:6] if type(x) == str else x)
    dfout['max'] = df.max().apply(lambda x: x[:6] if type(x) == str else x)
    dfout['nunique'] = df.apply(pd.Series.nunique)
    dfout['nan_count'] = nan_sum
    dfout['pct_nan'] = nan_sum / nrows
    
    return dfout.iloc[:nfeats, :]

In [78]:
custom_describe(options)

(369, 14)


,353,179,35,count,mean,std,min,25%,50%,75%,max,dtype,nunique,nan_count,pct_nan
Underlying,119.99,119.99,119.99,369,119.99,0.00,119.99,119.989998,119.989998,119.989998,119.99,float64,1,0,0.0
Ticker,AAPL,AAPL,AAPL,369,NaN,NaN,AAPL,NaN,NaN,NaN,AAPL,object,1,0,0.0
YTE,1.83836,0.0931507,0.0164384,369,0.40,0.55,0.0164384,0.054795,0.093151,0.572603,1.99178,float64,16,0,0.0
DTE,671,34,6,369,144.70,202.38,6,20.000000,34.000000,209.000000,727,int64,16,0,0.0
Strike,120,123,140,369,122.46,12.90,100,112.000000,122.000000,133.000000,149,float64,70,0,0.0
Last,22.4,3.63,0.04,369,9.38,8.59,0.01,1.460000,7.200000,16.190000,32.9,float64,304,0,0.0
Bid,22.35,3.5,0.03,369,9.13,8.37,0,1.370000,7.100000,15.450000,32.75,float64,284,0,0.0
Ask,22.55,3.6,0.04,369,9.26,8.45,0.02,1.450000,7.250000,15.650000,32.95,float64,291,0,0.0
IV,0.350379,0.330573,0.500005,369,0.37,0.05,0.312995,0.348151,0.353034,0.369879,0.601566,float64,285,0,0.0
NPV,0.0396267,1.25717,0.0231362,369,2.16,3.34,0.00853848,0.140765,0.552821,2.718925,18.4552,float64,369,0,0.0
